### 决策树

决策树，又称为"判定树"，是基于树结构来进行决策。决策树学习的目的是为
了产生一棵泛化能力强，即处理未见示例能力强的决策树。决策树学习基本算法过程如下:

![avatar](../image/机器学习_决策树_图1.jpg)

显然, 决策树的生成是一个递归过程. 在决策树基本算法中, 有三种情形会
导致递归返回:
* 当前结点包含的样本全属于同一类别, 无需划分;
* 当前属性集为空, 或是所有样本在所有属性上取值相同, 无法划分;
* 当前结点包含的样本集合为空, 不能划分。

**决策树学习的关键在于如何选择最优划分属性**。

### 划分选择

我们希望决策树的分支结点所包含的样本尽可能属于同一类别，即节点的"纯度"越来越高。

#### 信息增益
“信息嫡” (information entropy)是度量样本集合纯度最常用的一种指标。
假定当前样本集合 D 中第 $k$ 类样本所占的比例为 $p_{k}(k=1,2, \ldots,|\mathcal{Y}|),$ 则 $D$ 的信息嫡定义为
$$
\operatorname{Ent}(D)=-\sum_{k=1}^{|\mathcal{Y}|} p_{k} \log _{2} p_{k}
$$
$\operatorname{Ent}(D)$ 的值越小, 则 $D$ 的纯度越高.

假定离散属性 $a$ 有 $V$ 个可能的取值 $\left\{a^{1}, a^{2}, \ldots, a^{V}\right\},$ 若使用 $a$ 来对样本集
$D$ 进行划分，则会产生 $V$ 个分支结点, 其中第 $v$ 个分支结点包含了 $D$ 中所有在 属性 $a$ 上取值为 $a^{v}$ 的样本, 记为 $D^{v}$. 我们可根据式(4.1) 计算出 $D^{v}$ 的信息嫡, 再考虑到不同的分支结点所包含的样本数不同, 给分支结点赋予权重 | $D^{v}|/| D |$ 即样本数越多的分支结点的影响越大, 于是可计算出用属性 $a$ 对样本集 D 进行 划分所获得的“信息增益” (information gain)
$$
\operatorname{Gain}(D, a)=\operatorname{Ent}(D)-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \operatorname{Ent}\left(D^{v}\right)
$$
一般而言，信息增益越大, 则意味着使用属性 $a$ 来进行划分所获得的“纯 度提升”越大. 因此, 我们可用信息增益来进行决策树的划分属性选择, 即在选择最优划分属性时选择属性$a_{*}=\underset{a \in A}{\arg \max } \operatorname{Gain}(D, a)$。

实际上, **信息增益准则对可取值数目较多的属性有所偏好**，为減少这种
偏好可能带来的不利影响, 著名的C4.5决策树算法不直接使 用信息增益，而是使用“增益率” (gain ratio) 来选择最优划分属性. 增益率定义为
$$
\operatorname{Gain}_{ratio}(D, a)=\frac{\operatorname{Gain}(D, a)}{\operatorname{IV}(a)}
$$
其中
$$
\mathrm{IV}(a)=-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \log _{2} \frac{\left|D^{v}\right|}{|D|}
$$
称为属性a的"固有值"(intrinsic value).属性a的可能取值数目越多(即 V 越大), 则 IV $(a)$ 的值通常会越大. 

需注意的是, **增益率准则对可取值数目较少的属性有所偏好**, 因此, C4.5算法并不是直接选择增益率最大的候选划分属性, 而是使用了一个启发式
:先从候选划分属性中找出信息增益高于平均水平的属性, 再从中选择增益率最高的。

### 剪枝处理

剪枝是决策树学习算法对付“过拟合”的主要手段，决策树剪枝的基本策略有“预剪枝”和“后剪枝”。

预剪枝是指在决策树生成过程中, 对每个结点在划前先进行估计, 若当前结点的划分不能带来决策树泛化性能提升, 则停止划分并将当前结点标记为叶结点; 

后剪枝则是先从训练集生成一棵完整的决策树, 然后自底向上地对非叶结点进行考察, 若将该结点对应的子树替换为叶结点能带来决策树泛化性能提升, 则将该子树替换为叶结点。

我们可以预留一部分数据用作“验证集”以进行泛化性能的评估。

一般情形下, 后剪枝决策树的欠拟合风险很小，泛化性能往往优于预剪枝决策树。但后剪枝过程是在生成完全决策树之后进行的, 并且要自底向上地对树中的所有非叶结点进行逐一考察, 因此其训练时间开销比未剪枝决策树和预剪枝决策树都要大得多。